In [36]:
import pandas as pd

In [37]:
df_train = pd.read_csv('datasets/mnist_train.csv')
df_test = pd.read_csv('datasets/mnist_test.csv')

In [38]:
# separate x & y
x_train = df_train.drop(columns='label').values
y_train = df_train[['label']]/10

x_test = df_test.drop(columns='label').values
y_test = df_test[['label']]/10

In [39]:
# import PyTorch & related dependencies
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

In [40]:
# Define a network - this is handled in Keras with the Sequential() function
class Model(nn.Module):
    
    # Initialize model structure
    def __init__(self,input_dim):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(input_dim,256)
        self.layer2 = nn.Linear(256,128)
        self.layer3 = nn.Linear(128,10)
        
    # Logic for forward propagation
    def forward(self, x):
        x = F.relu(self.layer1(x)) # important that you apply these layers to the input (and each successive step)
        x = F.relu(self.layer2(x))
        x = F.softmax(self.layer3(x), dim=1) # calc softmax along axis 1
        return(x)

In [41]:
# Complile options
model = Model(x_train.shape[1])
optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

In [42]:
import numpy as np

import tqdm # visualization library for PyTorch

In [43]:
# Set up hyperparams
EPOCHS = 200
# Need to convert our input data into Torch objects
x_train = Variable(torch.from_numpy(np.array(x_train))).float()
x_test = Variable(torch.from_numpy(np.array(x_test))).float()
y_train = Variable(torch.from_numpy(np.array(y_train))).long() # long --> integer
y_test = Variable(torch.from_numpy(np.array(y_test))).long()

In [44]:
loss_list = []
accuracy_list_test = []
accuracy_list_train = []

In [45]:
# Iterate over epochs for each pass of the network
for epoch in tqdm.trange(EPOCHS):
    y_pred = model(x_train)
    loss = loss_fn(y_pred, y_train)
    loss_list.append(loss.item())
    correct_train = (torch.argmax(y_pred,dim=1)==y_train).type(torch.FloatTensor)
    accuracy_list_train.append(correct_train.mean())
    
    # zero-out gradient
    optimizer.zero_grad()
    # backward finds the derivative of the loss function (i.e. the new gradient)
    loss.backward()
    optimizer.step()
    
    # we do not want to calc new gradients on our validation data
    with torch.no_grad():
        y_test_pred = model(x_test)
        correct_test = (torch.argmax(y_test_pred,dim=1)==y_test).type(torch.FloatTensor)
        accuracy_list_test.append(correct_test.mean())

  0%|                                               | 0/200 [00:00<?, ?it/s]


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.figure(figsize=(12,6))
plt.subplot(2,1,1)
plt.plot(loss_list)
plt.title('Loss by Epoch')

plt.subplot(2,1,2)
plt.plot(accuracy_list_train,label="Train")
plt.plot(accuracy_list_test,label="Test")
plt.title('Accuracy by Epoch')
plt.legend()
plt.show()